# PCMCI+ Causal Discovery with Real Market Data

This notebook demonstrates how to use the high-performance PCMCI+ C library to discover causal relationships in financial time series.

**Features demonstrated:**
- Fetching market data with yfinance
- Volatility estimation (Parkinson, returns)
- PCMCI+ causal graph discovery
- CMI (Conditional Mutual Information) for nonlinear dependencies
- Distance Correlation for any-dependence detection
- Visualizing causal graphs

In [ ]:
# Install dependencies if needed
# !pip install yfinance pandas numpy matplotlib networkx

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

# Import PCMCI+ library
import pcmci

print(f"PCMCI+ version: {pcmci.version()}")

## 1. Fetch Market Data

We'll analyze cross-market relationships between:
- **SPY** - S&P 500 (US Equities)
- **TLT** - 20+ Year Treasury Bonds (Rates)
- **GLD** - Gold (Safe haven)
- **UUP** - US Dollar Index (Currency)
- **EEM** - Emerging Markets (Risk sentiment)
- **VIX** - Volatility Index (Fear gauge)

In [ ]:
# Define assets to analyze
ASSETS = {
    'SPY': 'US Equities',
    'TLT': 'Treasuries',
    'GLD': 'Gold',
    'UUP': 'US Dollar',
    'EEM': 'EM Equities',
}

# Fetch 2 years of daily data
end_date = datetime.now()
start_date = end_date - timedelta(days=2*365)

print(f"Fetching data from {start_date.date()} to {end_date.date()}...")

data = yf.download(
    list(ASSETS.keys()),
    start=start_date,
    end=end_date,
    progress=False
)

print(f"Downloaded {len(data)} days of data")
data.tail()

## 2. Compute Volatility

We'll use two volatility measures:
1. **Parkinson volatility** - High-low range estimator (more efficient than close-to-close)
2. **Realized volatility** - Rolling std of returns

In [ ]:
def parkinson_volatility(high, low, window=20):
    """Parkinson high-low volatility estimator (annualized)"""
    log_hl = np.log(high / low) ** 2
    factor = 1.0 / (4.0 * np.log(2))
    vol = np.sqrt(factor * log_hl.rolling(window).mean() * 252)
    return vol

def realized_volatility(close, window=20):
    """Rolling standard deviation of log returns (annualized)"""
    returns = np.log(close / close.shift(1))
    vol = returns.rolling(window).std() * np.sqrt(252)
    return vol

# Compute volatility for each asset
vol_window = 20
volatility = pd.DataFrame()

for symbol in ASSETS.keys():
    high = data['High'][symbol]
    low = data['Low'][symbol]
    close = data['Close'][symbol]
    
    # Use Parkinson volatility
    volatility[symbol] = parkinson_volatility(high, low, vol_window)

# Drop NaN rows
volatility = volatility.dropna()
print(f"Volatility series: {len(volatility)} observations")

volatility.tail()

In [ ]:
# Plot volatility time series
fig, ax = plt.subplots(figsize=(14, 6))

for symbol in ASSETS.keys():
    ax.plot(volatility.index, volatility[symbol], label=f"{symbol} ({ASSETS[symbol]})", alpha=0.8)

ax.set_ylabel('Annualized Volatility')
ax.set_title('Parkinson Volatility (20-day rolling)')
ax.legend(loc='upper right')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 3. Run PCMCI+ Causal Discovery

PCMCI+ will find:
- Which assets' volatility **causes** changes in other assets' volatility
- The **lag** at which this happens (e.g., TLT leads SPY by 2 days)
- The **strength** of the causal link (partial correlation)

In [ ]:
# Prepare data for PCMCI+
# Shape must be (n_vars, T)
var_names = list(ASSETS.keys())
vol_matrix = volatility[var_names].values.T  # Shape: (5, T)

print(f"Data shape: {vol_matrix.shape} (n_vars={vol_matrix.shape[0]}, T={vol_matrix.shape[1]})")

# Run PCMCI+
tau_max = 5  # Look for causal links up to 5 days lag
alpha = 0.05  # Significance level

print(f"\nRunning PCMCI+ with tau_max={tau_max}, alpha={alpha}...")

result = pcmci.run_pcmci(
    vol_matrix,
    tau_max=tau_max,
    alpha=alpha,
    var_names=var_names,
    use_spearman=True,  # Robust to outliers
    n_threads=0  # Auto-detect
)

print(f"\nCompleted in {result.runtime:.3f} seconds")
print(f"Found {result.n_significant} significant causal links")

In [ ]:
# Display results
print(result.summary())

In [ ]:
# Create a cleaner summary table
links_data = []
for link in result.significant_links:
    src = var_names[link.source_var]
    tgt = var_names[link.target_var]
    links_data.append({
        'Source': src,
        'Target': tgt,
        'Lag (days)': link.tau,
        'Strength': f"{link.val:.3f}",
        'p-value': f"{link.pval:.4f}",
        'Interpretation': f"{src} vol → {tgt} vol in {link.tau}d"
    })

links_df = pd.DataFrame(links_data)
if len(links_df) > 0:
    # Sort by absolute strength
    links_df['abs_strength'] = links_df['Strength'].astype(float).abs()
    links_df = links_df.sort_values('abs_strength', ascending=False).drop('abs_strength', axis=1)
    display(links_df)
else:
    print("No significant causal links found at alpha=0.05")

## 4. Visualize Causal Graph

In [ ]:
import networkx as nx

def plot_causal_graph(result, var_names, min_strength=0.0):
    """Plot causal graph using networkx"""
    G = nx.DiGraph()
    
    # Add nodes
    for name in var_names:
        G.add_node(name)
    
    # Add edges
    edge_labels = {}
    edge_colors = []
    edge_widths = []
    
    for link in result.significant_links:
        if abs(link.val) < min_strength:
            continue
        if link.tau == 0 and link.source_var == link.target_var:
            continue  # Skip self-loops at lag 0
            
        src = var_names[link.source_var]
        tgt = var_names[link.target_var]
        
        G.add_edge(src, tgt)
        edge_labels[(src, tgt)] = f"τ={link.tau}\nr={link.val:.2f}"
        edge_colors.append('green' if link.val > 0 else 'red')
        edge_widths.append(abs(link.val) * 5)
    
    if len(G.edges()) == 0:
        print("No edges to display")
        return
    
    # Plot
    fig, ax = plt.subplots(figsize=(12, 8))
    
    pos = nx.spring_layout(G, k=2, iterations=50, seed=42)
    
    # Draw nodes
    nx.draw_networkx_nodes(G, pos, node_size=3000, node_color='lightblue', 
                          edgecolors='black', linewidths=2, ax=ax)
    nx.draw_networkx_labels(G, pos, font_size=12, font_weight='bold', ax=ax)
    
    # Draw edges
    nx.draw_networkx_edges(G, pos, edge_color=edge_colors, width=edge_widths,
                          arrows=True, arrowsize=25, arrowstyle='-|>',
                          connectionstyle='arc3,rad=0.1', ax=ax)
    
    # Edge labels
    nx.draw_networkx_edge_labels(G, pos, edge_labels, font_size=9, ax=ax)
    
    ax.set_title('Volatility Spillover Graph (PCMCI+)\nGreen=positive, Red=negative', fontsize=14)
    ax.axis('off')
    plt.tight_layout()
    plt.show()

plot_causal_graph(result, var_names, min_strength=0.05)

## 5. Compare Independence Tests

Let's compare different tests on pairs of assets:
- **Partial Correlation** - Fast, linear only
- **CMI** - Detects nonlinear dependencies
- **Distance Correlation** - Detects any dependence

In [ ]:
# Compare independence tests on asset pairs
def compare_tests(X, Y, name_x, name_y):
    """Compare different independence tests"""
    print(f"\n{'='*60}")
    print(f"Testing: {name_x} vs {name_y}")
    print('='*60)
    
    # Partial correlation
    r, p = pcmci.parcorr_test(X, Y)
    print(f"Partial Correlation: r={r:+.4f}, p={p:.4f}")
    
    # CMI
    result = pcmci.cmi_test(X, Y, n_perm=100)
    print(f"CMI (KSG k=5):       cmi={result.cmi:.4f}, p={result.pvalue:.4f}")
    
    # Distance correlation
    result = pcmci.dcor_test(X, Y, n_perm=100)
    print(f"Distance Corr:       dcor={result.dcor:.4f}, p={result.pvalue:.4f}")

# Test a few pairs
vol_arr = volatility.values

compare_tests(vol_arr[:, 0], vol_arr[:, 1], 'SPY', 'TLT')
compare_tests(vol_arr[:, 0], vol_arr[:, 2], 'SPY', 'GLD')
compare_tests(vol_arr[:, 0], vol_arr[:, 4], 'SPY', 'EEM')
compare_tests(vol_arr[:, 1], vol_arr[:, 2], 'TLT', 'GLD')

## 6. Conditional Independence Test

Test if two assets are independent **after controlling for** a third asset.

Example: Are SPY and EEM independent given GLD?

In [ ]:
def conditional_test(X, Y, Z, name_x, name_y, name_z):
    """Test X ⊥ Y | Z"""
    print(f"\n{'='*60}")
    print(f"Testing: {name_x} ⊥ {name_y} | {name_z}")
    print('='*60)
    
    # Unconditional first
    r, p = pcmci.parcorr_test(X, Y)
    print(f"Unconditional parcorr({name_x}, {name_y}): r={r:+.4f}, p={p:.4f}")
    
    # Conditional
    r, p = pcmci.parcorr_test(X, Y, Z)
    print(f"Conditional parcorr({name_x}, {name_y} | {name_z}): r={r:+.4f}, p={p:.4f}")
    
    # CMI conditional
    result_uncond = pcmci.cmi_test(X, Y, n_perm=100)
    result_cond = pcmci.cmi_test(X, Y, Z, n_perm=100)
    print(f"\nCMI({name_x}; {name_y}):       {result_uncond.cmi:.4f}, p={result_uncond.pvalue:.4f}")
    print(f"CMI({name_x}; {name_y} | {name_z}): {result_cond.cmi:.4f}, p={result_cond.pvalue:.4f}")

# Test conditional independence
spy = vol_arr[:, 0]
tlt = vol_arr[:, 1]
gld = vol_arr[:, 2]
uup = vol_arr[:, 3]
eem = vol_arr[:, 4]

conditional_test(spy, eem, gld, 'SPY', 'EEM', 'GLD')
conditional_test(spy, gld, tlt, 'SPY', 'GLD', 'TLT')
conditional_test(tlt, gld, uup, 'TLT', 'GLD', 'UUP')

## 7. Lead-Lag Analysis

Which asset's volatility **leads** which? This is the core question for spillover detection.

In [ ]:
def lead_lag_analysis(source, target, name_src, name_tgt, max_lag=10):
    """Analyze lead-lag relationship between two assets"""
    results = []
    
    for lag in range(0, max_lag + 1):
        if lag == 0:
            x = source
            y = target
        else:
            x = source[:-lag]  # Source at t-lag
            y = target[lag:]   # Target at t
        
        r, p = pcmci.parcorr_test(x, y)
        results.append({'lag': lag, 'corr': r, 'pvalue': p})
    
    df = pd.DataFrame(results)
    
    # Plot
    fig, ax = plt.subplots(figsize=(10, 4))
    
    colors = ['green' if p < 0.05 else 'gray' for p in df['pvalue']]
    ax.bar(df['lag'], df['corr'], color=colors, alpha=0.7, edgecolor='black')
    
    ax.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
    ax.set_xlabel('Lag (days)')
    ax.set_ylabel('Correlation')
    ax.set_title(f'Lead-Lag: {name_src}(t-lag) → {name_tgt}(t)\nGreen = significant (p<0.05)')
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    # Find optimal lag
    sig_results = df[df['pvalue'] < 0.05]
    if len(sig_results) > 0:
        best = sig_results.loc[sig_results['corr'].abs().idxmax()]
        print(f"Strongest link: {name_src}(t-{int(best['lag'])}) → {name_tgt}(t), r={best['corr']:.3f}")
    else:
        print("No significant lead-lag relationship found")

# Analyze lead-lag for interesting pairs
lead_lag_analysis(tlt, spy, 'TLT', 'SPY')
lead_lag_analysis(spy, eem, 'SPY', 'EEM')
lead_lag_analysis(uup, gld, 'UUP', 'GLD')

## 8. Returns Analysis (Alternative to Volatility)

We can also analyze causal relationships in returns instead of volatility.

In [ ]:
# Compute log returns
returns = pd.DataFrame()
for symbol in ASSETS.keys():
    close = data['Close'][symbol]
    returns[symbol] = np.log(close / close.shift(1))

returns = returns.dropna()
print(f"Returns: {len(returns)} observations")

# Run PCMCI+ on returns
returns_matrix = returns.values.T

print(f"\nRunning PCMCI+ on returns...")
result_returns = pcmci.run_pcmci(
    returns_matrix,
    tau_max=5,
    alpha=0.05,
    var_names=var_names,
    use_spearman=True
)

print(f"Completed in {result_returns.runtime:.3f} seconds")
print(f"Found {result_returns.n_significant} significant causal links in returns")

In [ ]:
# Compare volatility vs returns causal structures
print("=" * 60)
print("VOLATILITY Causal Links:")
print("=" * 60)
for link in result.significant_links:
    if link.tau > 0 or link.source_var != link.target_var:
        src = var_names[link.source_var]
        tgt = var_names[link.target_var]
        print(f"  {src}(t-{link.tau}) → {tgt}(t): r={link.val:+.3f}")

print("\n" + "=" * 60)
print("RETURNS Causal Links:")
print("=" * 60)
for link in result_returns.significant_links:
    if link.tau > 0 or link.source_var != link.target_var:
        src = var_names[link.source_var]
        tgt = var_names[link.target_var]
        print(f"  {src}(t-{link.tau}) → {tgt}(t): r={link.val:+.3f}")

## 9. Performance Benchmark

In [ ]:
import time

print("Performance Benchmark")
print("=" * 60)

# Test with different data sizes
n_vars = 5
for T in [100, 250, 500, 1000]:
    data_test = np.random.randn(n_vars, T)
    
    start = time.time()
    result = pcmci.run_pcmci(data_test, tau_max=3, alpha=0.05)
    elapsed = time.time() - start
    
    print(f"n_vars={n_vars}, T={T:4d}: {elapsed*1000:6.1f} ms")

print()

# Test independence tests
X = np.random.randn(1000)
Y = np.random.randn(1000)

start = time.time()
for _ in range(100):
    pcmci.parcorr_test(X, Y)
print(f"parcorr_test (1000 samples, 100 runs): {(time.time()-start)*10:.2f} ms/call")

start = time.time()
pcmci.mi(X, Y)
print(f"MI (1000 samples): {(time.time()-start)*1000:.2f} ms")

start = time.time()
pcmci.dcor(X, Y)
print(f"dCor (1000 samples): {(time.time()-start)*1000:.2f} ms")

## 10. Summary

### What we learned:

1. **PCMCI+** discovers causal relationships with time lags in multivariate time series
2. **Lead-lag detection**: Identifies which asset moves first and predicts others
3. **Multiple tests available**:
   - `parcorr_test`: Fast, linear relationships
   - `cmi_test`: Nonlinear dependencies (KSG estimator)
   - `dcor_test`: Any dependence detection
   - `gpdc_test`: Nonlinear conditional independence

### Performance:
- Sub-millisecond for small problems
- ~100ms for realistic trading applications (5-10 vars, 500 samples)
- Parallelized with OpenMP

### Use cases:
- **Volatility spillover**: Which market's stress predicts another?
- **Lead-lag trading**: Exploit predictive relationships
- **Risk management**: Understand contagion pathways
- **Feature engineering**: Identify causal predictors for ML models